In [3]:
import requests
from requests import Response
from pydantic import RootModel,BaseModel,Field,field_validator
from datetime import datetime
import pandas as pd 

class Site (BaseModel):
    行政區域:str = Field(alias='sarea')
    總數量:int = Field(alias='total')
    可借數量:int = Field(alias='available_rent_bikes') 
    可還數量:int = Field(alias='available_return_bikes')
    時間:datetime = Field(alias='mday')

    @field_validator('可借數量','可還數量',mode='before')
    @classmethod
    def whitespace_to_zero(cls, value: str) -> str:
        return '0.0' if value == '' else value
    
class Youbike(RootModel):
    root:list[Site]

try:
    youbike_url:Response= requests.get('https://tcgbusfs.blob.core.windows.net/dotapp/youbike/v2/youbike_immediate.json')
    youbike_url.raise_for_status()
except Exception as e:
    print(e)
else:
    data = Youbike.model_validate_json(youbike_url.text)
    all_sites = data.model_dump()

df = pd.DataFrame(all_sites)
df

,行政區域,總數量,可借數量,可還數量,時間
0,大安區,28,8,20,2024-07-02 16:39:22
1,大安區,21,2,19,2024-07-02 16:38:23
2,大安區,16,3,13,2024-07-02 16:33:20
3,大安區,11,5,6,2024-07-02 16:38:23
4,大安區,16,1,15,2024-07-02 16:37:21
...,...,...,...,...,...
1425,臺大公館校區,30,13,5,2024-07-02 16:39:22
1426,臺大公館校區,20,19,1,2024-07-02 16:37:21
1427,臺大公館校區,24,8,8,2024-07-02 16:37:21
1428,臺大公館校區,40,27,4,2024-07-02 16:37:21


In [2]:
df['可借數量']

0        9
1        1
2        1
3        3
4        1
        ..
1425     9
1426    19
1427     8
1428    31
1429     5
Name: 可借數量, Length: 1430, dtype: int64

In [4]:
#將數值分入離散區間。
#此函數對於從連續變數到分類變數也很有用。例如，cut可以將年齡轉換為年齡範圍組。支援分入相同數量的分箱或預先指定的分箱陣列。
#pandas.cut(x, bins, right=True, labels=None, retbins=False, precision=3, include_lowest=False, duplicates='raise', ordered=True)

bins = [0,1,4,7,11,100]
labels = ['0輛','1~3輛','4~6輛','7~10輛','10輛以上']

df['可借'] = pd.cut(df['可借數量'],bins=bins,right=False,labels=labels) ##right=False 的意思就是不包含 右邊的數字

df

,行政區域,總數量,可借數量,可還數量,時間,可借
0,大安區,28,10,18,2024-07-02 16:22:23,7~10輛
1,大安區,21,4,17,2024-07-02 16:23:16,4~6輛
2,大安區,16,4,12,2024-07-02 16:17:44,4~6輛
3,大安區,11,3,8,2024-07-02 16:19:24,1~3輛
4,大安區,16,2,14,2024-07-02 16:23:16,1~3輛
...,...,...,...,...,...,...
1425,臺大公館校區,30,11,7,2024-07-02 16:21:22,10輛以上
1426,臺大公館校區,20,20,0,2024-07-02 16:13:18,10輛以上
1427,臺大公館校區,24,9,7,2024-07-02 16:19:24,7~10輛
1428,臺大公館校區,40,31,0,2024-07-02 16:21:15,10輛以上


In [5]:
df1=df.groupby(by=['行政區域','可借'])[['可借數量']].count()
df1.rename(columns={'可借數量':'站點數量'})

/tmp/ipykernel_30663/4053175281.py:1: FutureWarning: The default of observed=False is deprecated and will be changed to True in a future version of pandas. Pass observed=False to retain current behavior or observed=True to adopt the future default and silence this warning.
  df1 = df.groupby(by=['行政區域','可借'])[['可借數量']].count()


站點數量
行政區域 可借         
中山區  0輛       13
     1~3輛     28
     4~6輛     24
     7~10輛    38
     10輛以上    53
...          ...
萬華區  0輛        7
     1~3輛     21
     4~6輛     13
     7~10輛     9
     10輛以上    30

[65 rows x 1 columns]

NameError: name 'df' is not defined